In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import glob
import cv2

In [ ]:
# For smaller dataset (100 images of masked and non-masked)

#pathNoMask = glob.glob("/content/drive/My Drive/APS360/Project Files/dataset_final/dataset_final/nomask/*.png")

#pathMask = glob.glob("/content/drive/My Drive/APS360/Project Files/dataset_final/dataset_final/mask/*.png")

In [4]:
# For full dataset

pathNoMask = glob.glob("/content/drive/My Drive/APS360/APS360 Project/dataset_final/nomask/*.png")

pathMask = glob.glob("/content/drive/My Drive/APS360/APS360 Project/dataset_final/mask/*.png")

In [6]:
imagesNoMask = []

for imgPath in pathNoMask:
    img=cv2.imread(str(imgPath))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)       
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)           
    img=cv2.resize(img,(224,224))
    img = np.array(img)
    img = img.flatten()
    imagesNoMask.append(img)

imagesNoMask = np.array(imagesNoMask)


In [7]:
imagesMask = []
for imgPath in pathMask:
    img=cv2.imread(str(imgPath))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)       
    #img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)     
    img=cv2.resize(img,(224,224))
    img = np.array(img)
    img = img.flatten()
    imagesMask.append(img)

imagesMask = np.array(imagesMask)


In [8]:
#labelsNoMask = [0]*100
labelsNoMask = [0]*3000
labelsNoMask = np.array(labelsNoMask)
#labelsMask = [1]*100
labelsMask = [1]*3000
labelsMask = np.array(labelsMask)

In [9]:
#training_data = np.concatenate((imagesMask[0:80],imagesNoMask[0:80]), axis=0)
#training_labels = np.concatenate((labelsMask[0:80],labelsNoMask[0:80]), axis=0)

training_data = np.concatenate((imagesMask[0:2400],imagesNoMask[0:2400]), axis=0)
training_labels = np.concatenate((labelsMask[0:2400],labelsNoMask[0:2400]), axis=0)

print(training_data)
print(training_labels)
#print(training_labels.shape)

[[ 78  86  87 ...  17  21  27]
 [ 10  10  13 ...  37  40  40]
 [ 43  41  35 ... 184 189 191]
 ...
 [218 218 218 ... 169 169 169]
 [ 88  88  90 ...   2   3   3]
 [ 10  12  21 ...  38  38  38]]
[1 1 1 ... 0 0 0]


In [10]:
from random import shuffle

#ind_list = [i for i in range(160)] 160 (= 100*80%*2) total images for training, 
ind_list = [i for i in range(4800)]
shuffle(ind_list)
training_data = training_data[ind_list, :]
training_labels = training_labels[ind_list,]

print(training_data)
print(training_labels)


[[ 36  35  32 ...  82  69  63]
 [ 91  82  77 ... 160 161 163]
 [ 47  47  53 ...  10  10  10]
 ...
 [238 238 238 ... 238 233 232]
 [ 15  15  15 ...   2   2   2]
 [175 175 176 ...  19  20  20]]
[0 0 1 ... 1 1 0]


In [11]:
#testing_data = np.concatenate((imagesMask[80:],imagesNoMask[80:]), axis=0)
#testing_labels = np.concatenate((labelsMask[80:],labelsNoMask[80:]), axis=0)

testing_data = np.concatenate((imagesMask[2400:],imagesNoMask[2400:]), axis=0)
testing_labels = np.concatenate((labelsMask[2400:],labelsNoMask[2400:]), axis=0)

print(testing_data.shape)
print(testing_labels)
#print(training_labels.shape)

(1200, 50176)
[1 1 1 ... 0 0 0]


In [ ]:
# overfit check
# Only run this if you want to check overfit
testing_data = training_data 
testing_labels = training_labels

In [12]:
# Support Vector Machines
from sklearn import svm
model = svm.SVC(gamma=2, C=1)


In [13]:
# Fit the model to our training data
model.fit(training_data, training_labels)

# Make predictions
testing_predicted = model.predict(testing_data)
score = 100*(1-sum(abs(testing_predicted-testing_labels))/len(testing_predicted))
print("SVM Test:", score)

SVM Test: 56.00000000000001


In [14]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=250)

# Fit the model to our training data
model.fit(training_data, training_labels)

# Make predictions
testing_predicted = model.predict(testing_data)
score = 100*(1-sum(abs(testing_predicted-testing_labels))/len(testing_predicted))
print("RF Test:", score)

RF Test: 81.16666666666667


In [15]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# Fit the model to our training data
model.fit(training_data, training_labels)

# Make predictions
testing_predicted = model.predict(testing_data)
score = 100*(1-sum(abs(testing_predicted-testing_labels))/len(testing_predicted))
print("NB Test:", score)


NB Test: 67.83333333333333


In [16]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=15)

# Fit the model to our training data
model.fit(training_data, training_labels)

# Make predictions
testing_predicted = model.predict(testing_data)
score = 100*(1-sum(abs(testing_predicted-testing_labels))/len(testing_predicted))
print("DT Test:", score)

DT Test: 73.25
